$\Delta u + k^2(1+q) u = f $ in $\Omega = [0,1]^2$    
$u = 0 $ on $\partial \Omega$

In [1]:
import scipy as sp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres
import time

In [2]:
k = 1  # wavenumber
N = 50  # 格点数
h = 1 / N  # 间隔

In [3]:
def q_gen_example(N):
    q = np.zeros((N + 1, N + 1))
    q_value = 0.02
    x1, x2, x3, y1, y2, y3, y4 = 0.2, 0.4, 0.7, 0.2, 0.3, 0.6, 0.7
    q[int(x1 * N):int(x2 * N), int(y1 * N):int(y4 * N)] = q_value
    q[int(x2 * N):int(x3 * N), int(y2 * N):int(y3 * N)] = q_value
    return q

def q_gen_1(N,b1 = 0.3,b2 = 0.6,a1 = 8,a2 = 9,gamma = 1): # 𝑞(𝑥,𝑦)= 𝜆   exp⁡(−𝑎1(𝑥−𝑏1 )^2−a2 (y−b2 )^2 ) 

    q = np.zeros((N + 1, N + 1))
    for i in range(1,N):
        for j in range(1,N):
            q[i,j] = gamma* np.exp(-a1*(i/N - b1)**2-a2*(j/N - b2)**2)
    return q

def q_generation(N, method='T'): 
    if method == 'T':
        return q_gen_example(N)
    elif method == 'Gauss':
        return q_gen_1(N)
    print('method error')


# q = q_generation(N)
# sns.heatmap(q, xticklabels=False, yticklabels=False)

$u = \sin(x\pi)\sin(y\pi)$  
$f = \Delta u + (1+q) u = (1+q-2\pi^2)u$

In [4]:
def u_gen(N):
    u = np.zeros((N+1,N+1))
    for i in range(1,N):
        for j in range(1,N):
            u[i,j] = np.sin(i*np.pi/N)*np.sin(j*np.pi/N)
    return u

# u_truth = u_gen(N)
# sns.heatmap(u_truth, xticklabels=False, yticklabels=False)

In [5]:
def f_gen_1(N,q,u,k = 1):
    return ((1+q)*k*k-2*np.pi*np.pi)*u

# f = f_gen_1(N,q,u_truth)
# sns.heatmap(f, xticklabels=False, yticklabels=False)

### Method1 五点格式
$(u_{i+1,j} +u_{i-1,j} +u_{i,j+1} +u_{i,j-1} - 4u_{i,j})/h^2 + (1+q_{i,j})u_{i,j} = f_{i,j}$  
$M_1 = \text{Tri}\,(T_1,T_1,T_2,\cdots,T_{N-1};I,I,\cdots,I;I,I,\cdots,I)$  
$T_{i} = \text{Tri}\,(1+q_{i,1}-\frac{4}{h^2},1+q_{i,2}-\frac{4}{h^2},\cdots,1+q_{i,n}-\frac{4}{h^2};\frac{1}{h^2},\frac{1}{h^2},\cdots,\frac{1}{h^2};\frac{1}{h^2},\frac{1}{h^2},\cdots,\frac{1}{h^2})$  
$U = (u_{1,1},u_{1,2},\ldots ,u_{n,n})^T$  
$F_1 = (f_{1,1},f_{1,2},\ldots ,f_{n,n})^T$  
$M_1U = F$

In [6]:
def Matrix_5(N, q,k = 1):
    M = N - 1
    Q = q[1:-1, 1:-1].reshape(1, -1)
    data1 = k*k*(1+Q)-4*N*N
    data2_minus = N*N*np.tile(np.append(np.ones(M-1),0),M).reshape(1,-1)
    data2_plus = N*N*np.tile(np.insert(np.ones(M-1),0,0),M).reshape(1,-1)
    data3 = N*N*np.ones(M*M).reshape(1,-1)
    data = np.r_[data1,data2_minus,data2_plus,data3,data3]
    offsets = np.array([0, -1, 1,-M,M])
    dia = sp.sparse.dia_matrix((data, offsets), shape=(M*M, M*M))
    return dia
    
#     Q = q[1:-1, 1:-1].reshape(-1, 1)
#     row, col, data = np.array([]), np.array([]), np.array([])
#     for i in range(M * M):
#         row = np.append(row, i)
#         col = np.append(col, i)
#         data = np.append(data, 1 + Q[i] - 4 * N * N)
#         if (i + 1) % M != 0:
#             row = np.append(row, i)
#             col = np.append(col, i + 1)
#             data = np.append(data, N * N)
#         if (i + M) < M * M:
#             row = np.append(row, i)
#             col = np.append(col, i + M)
#             data = np.append(data, N * N)
#         if i % M != 0:
#             row = np.append(row, i)
#             col = np.append(col, i - 1)
#             data = np.append(data, N * N)
#         if i - M > -1:
#             row = np.append(row, i)
#             col = np.append(col, i - M)
#             data = np.append(data, N * N)
#     return csc_matrix((data, (row, col)), shape=(M * M, M * M))

### Method2 九点格式
$A \,u_{i,j} = u_{i+1,j} +u_{i-1,j} +u_{i,j+1} +u_{i,j-1}  $  
$\frac{A-4I}{h^2} u_{i,j} + k^2(1+q_{i,j})u_{i,j} = f_{i,j} $ 
$B \,u_{i,j} = u_{i+1,j+1} +u_{i-1,j-1} +u_{i-1,j+1} +u_{i+1,j-1}$  
$\frac{A-4I}{h^2} u_{i,j} + \frac{B - 2A+4I}{6h^2}u_{i,j} + k^2(1+q_{i,j})u_{i,j} + \frac{k^2}{12}(A-4I)
(1+q_{i,j})u_{i,j}= f_{i,j} + \frac{1}{12}(A-4I)f_{i,j}$  
$\Rightarrow\quad (B+4A-20I) u_{i,j} +h^2 k^2(0.5A+4I)
(1+q_{i,j})u_{i,j}= h^2 (0.5A+4I)f_{i,j}$

In [7]:
def A(v):
    v[0, :] = 0
    v[-1, :] = 0
    v[:, 0] = 0
    v[:, -1] = 0
    return v[1:-1, 2:] + v[:-2, 1:-1] + v[2:, 1:-1] + v[1:-1, :-2]


def Matrix_9(N, q, k=1):
    M = N - 1
    h = 1 / N

    Q = q[1:-1, 1:-1].reshape(1, -1)

    value_1 = (1 + Q) * h * h * k * k * 4 - 20  # 主对角线
    value_2 = 4 + 0.5 * h * h * k * k * (1 + q[1:-1, 1:-1])  # A 三对角线&主对角元三对角线

    data_main = value_1  # 主对角线
    data_main_up = np.reshape(np.c_[np.zeros((M, 1)), value_2[:, :-1]],
                              (1, -1))  # 主对角元上对角线
    data_main_down = np.reshape(np.c_[value_2[:, 1:],
                                      np.zeros((M, 1))], (1, -1))  # 主对角元下对角线
    data_up = np.c_[np.zeros((1, M)), value_2[:-1, :].reshape(1,
                                                              -1)]  # 上对角元对角线
    data_up_up = np.tile(np.insert(np.ones(M - 1), 0, 0),
                         M).reshape(1, -1)  # 上对角元上对角线
    data_up_down = np.tile(np.append(np.ones(M - 1), 0),
                           M).reshape(1, -1)  # 上对角元下对角线
    data_down = np.c_[value_2[1:, :].reshape(1, -1), np.zeros((1, M))]  # 下对角元
    data_down_up = data_up_up  # 下对角元上对角线
    data_down_down = data_up_down  # 下对角元下对角线
    data = np.r_[data_main, data_main_up, data_main_down, data_up, data_up_up,
                 data_up_down, data_down, data_down_up, data_down_down]
    offsets = np.array([0, 1, -1, M, M + 1, M - 1, -M, -M + 1, -M - 1])
    dia = sp.sparse.dia_matrix((data, offsets), shape=(M * M, M * M))
    return dia


#     Q = q[1:-1, 1:-1].reshape(M * M, 1)
#     row, col, data = np.array([]), np.array([]), np.array([])
#     value_1 = (1 + Q) * h * h * k * k * 4 - 20  # 主对角线
#     value_2 = 4 + 0.5 * h * h * k * k * (1 + Q)  # A 三对角线&主对角元三对角线
#     for i in range(M * M):
#         row = np.append(row, i)
#         col = np.append(col, i)
#         data = np.append(data, value_1[i])  # 主对角线
#         if (i + M) < M * M:
#             row = np.append(row, i)
#             col = np.append(col, i + M)
#             data = np.append(data, value_2[i])  # 副对角线
#             if (i + 1) % M != 0:
#                 row = np.append(row, i)
#                 col = np.append(col, i + M + 1)
#                 data = np.append(data, 1)
#             if i % M != 0:
#                 row = np.append(row, i)
#                 col = np.append(col, i + M - 1)
#                 data = np.append(data, 1)
#         if i - M > -1:
#             row = np.append(row, i)
#             col = np.append(col, i - M)
#             data = np.append(data, value_2[i])
#             if (i + 1) % M != 0:
#                 row = np.append(row, i)
#                 col = np.append(col, i - M + 1)
#                 data = np.append(data, 1)
#             if i % M != 0:
#                 row = np.append(row, i)
#                 col = np.append(col, i - M - 1)
#                 data = np.append(data, 1)
#         if (i + 1) % M != 0:
#             row = np.append(row, i)
#             col = np.append(col, i + 1)
#             data = np.append(data, value_2[i])
#         if i % M != 0:
#             row = np.append(row, i)
#             col = np.append(col, i - 1)
#             data = np.append(data, value_2[i])
#     return csc_matrix((data, (row, col)), shape=(M * M, M * M))

In [8]:
def perform(N, q_method = 'T',k = 1,tol=1e-05, restart=20):
    def Error(a, a_truth, gap=1e-10):
        a1 = np.where(a < gap, gap, a)
        a_t1 = np.where(a_truth < gap, gap, a_truth)
        return np.abs(a1 / a_t1 - 1)

    q = q_generation(N,q_method)
    u_truth = u_gen(N)
    f = f_gen_1(N, q, u_truth,k = k)
    h = 1 / N

    time50 = time.time()
    Matrix5 = Matrix_5(N, q,k= k)
    Right5 = f[1:-1, 1:-1].reshape((-1, 1))
    time51 = time.time() - time50
    u_res, exit = gmres(Matrix5, Right5, tol=tol, restart=restart)
    time52 = time.time() - time50 - time51
    if exit == 0:
        res5 = np.zeros((N + 1, N + 1))
        res5[1:-1, 1:-1] = u_res.reshape(N - 1, N - 1)
        err5 = np.linalg.norm(Error(res5, u_truth), ord=2) / (N - 1)
    else:
        print('五点格式不收敛')
    

    time90 = time.time()
    Matrix9 = Matrix_9(N, q,k = k)
    Right9 = ((0.5 * A(f) + 4 * f[1:-1, 1:-1]) * h * h).reshape((-1, 1))
    time91 = time.time() - time90
    u_res, exit = gmres(Matrix9, Right9, tol=tol, restart=restart)
    time92 = time.time() - time90 - time91
    if exit == 0:
        res9 = np.zeros((N + 1, N + 1))
        res9[1:-1, 1:-1] = u_res.reshape(N - 1, N - 1)
        err9 = np.linalg.norm(Error(res9, u_truth), ord=2) / (N - 1)
    else:
        print('九点格式不收敛')
    

    print('N = %d' % N)
    print('五点格式平均相对误差为%f,生成矩阵用时%f,求解矩阵用时%f' % (err5, time51, time52))
    print('九点格式平均相对误差为%f,生成矩阵用时%f,求解矩阵用时%f' % (err9, time91, time92))

In [9]:
perform(100)

N = 100
五点格式平均相对误差为0.000085,生成矩阵用时0.001628,求解矩阵用时0.124833
九点格式平均相对误差为0.000005,生成矩阵用时0.001311,求解矩阵用时0.068261


In [10]:
perform(50)

N = 50
五点格式平均相对误差为0.000347,生成矩阵用时0.000723,求解矩阵用时0.004870
九点格式平均相对误差为0.000004,生成矩阵用时0.000566,求解矩阵用时0.003524


In [11]:
perform(200)

N = 200
五点格式平均相对误差为0.000013,生成矩阵用时0.002316,求解矩阵用时0.395872
九点格式平均相对误差为0.000007,生成矩阵用时0.002598,求解矩阵用时0.310352


In [12]:
perform(200,tol=1e-03)

N = 200
五点格式平均相对误差为0.000252,生成矩阵用时0.000854,求解矩阵用时0.058737
九点格式平均相对误差为0.000259,生成矩阵用时0.002142,求解矩阵用时0.066536


In [13]:
perform(200,restart=100)

N = 200
五点格式平均相对误差为0.000023,生成矩阵用时0.000722,求解矩阵用时0.467758
九点格式平均相对误差为0.000004,生成矩阵用时0.002012,求解矩阵用时0.403278


In [14]:
perform(200,k = 2)

N = 200
五点格式平均相对误差为0.000016,生成矩阵用时0.000668,求解矩阵用时1.482639
九点格式平均相对误差为0.000010,生成矩阵用时0.002719,求解矩阵用时1.150405


In [15]:
perform(200,q_method='Gauss')

N = 200
五点格式平均相对误差为0.000012,生成矩阵用时0.000597,求解矩阵用时1.079887
九点格式平均相对误差为0.000010,生成矩阵用时0.001434,求解矩阵用时0.938977


* 现在时间主要花在生成右矩阵上，原来逐点定义的左矩阵的值，现在按对角线定义极大节省了时间
* 生成矩阵和求解矩阵的用时都随着N的增加而高阶地变化
* 九点格式误差明显低于五点格式的矩阵方法
* 更改求解方程时的误差(tol)和重启数值(restart)可以调整求解用时
* 增加k的值会使求解矩阵的用时增加，可以通过等效改变q的值实现
* q的分布为Gauss型时求解矩阵的用时会增加